In [1]:
import pandas as pd
import numpy as np
import os
from nice_funcs.indicators import EWMA,Normalize,MACD

In [2]:
path_diario = './assets/1d/'
ativos = os.listdir(path_diario)

In [3]:
ativosOHLC = {}
for ativo in ativos:
    ativosOHLC[ativo.replace('.xlsx','')] = \
        pd.read_excel(os.path.join(path_diario,ativo),index_col=0)
    

In [4]:
close_prices = {}
for k in ativosOHLC.keys():
  close_prices[k] = ativosOHLC[k].Close


In [5]:
df_fechamento = pd.DataFrame(close_prices)
df_fechamento_normalized = df_fechamento.apply(lambda row: Normalize(row))

In [6]:
macd_signal = df_fechamento_normalized.apply(lambda row: MACD(row)[-1])
macd_original = df_fechamento_normalized.apply(lambda row: MACD(row)[0])
df_fechamento_normalized_ewma_5 = df_fechamento.apply(lambda row: EWMA(row,5))
df_fechamento_normalized_ewma_20 = df_fechamento.apply(lambda row: EWMA(row,20))
diff_ewma20_ewma5 = (df_fechamento_normalized_ewma_20 - df_fechamento_normalized_ewma_5)

In [7]:
diff_ewma20_ewma5

,ADAUSDT,BNBUSDT,BTCUSDT,ETHUSDT,LTCUSDT,XRPUSDT
Date,,,,,,
2018-05-04,NaN,NaN,NaN,NaN,NaN,NaN
2018-05-05,NaN,NaN,NaN,NaN,NaN,NaN
2018-05-06,NaN,NaN,NaN,NaN,NaN,NaN
2018-05-07,NaN,NaN,NaN,NaN,NaN,NaN
2018-05-08,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2023-07-31,-0.001554,0.552599,320.831168,12.370699,0.271747,-0.017802
2023-08-01,-0.001406,-0.991611,193.863271,9.169120,-0.076289,-0.016049
2023-08-02,0.001347,-0.224892,234.929832,14.939782,1.240016,-0.009268


In [8]:
import gymnasium
from ddpg_tf2 import Agent

In [13]:
env = gymnasium.make('Pendulum-v1')

In [31]:
agent = Agent(input_dims = env.observation_space.shape[0],\
   env = env, n_actions =env.action_space.shape[0])

n_games = 250
figure_file = 'plots/pendulum.png'
best_score = env.reward_range[0]
score_history = []
load_checkpoint = False

if load_checkpoint:
  n_steps = 0
  while n_steps <= agent.batch_size:
    observation, info = env.reset()
    action = env.action_space.sample()
    observation_, reward, _, done ,info = env.step(action)
    agent.remember(observation, action, reward, observation_, done)
    n_steps +=1
    agent.learn()
    agent.load_models()
    evaluate = True
else:
  evaluate = False

for i in range(n_games):
  observation, info = env.reset()
  done = False
  score = 0
  while not done:
    #print(score)
    action = agent.choose_action(observation, evaluate)
    observation_, reward, _ ,done ,info = env.step(action)
    score += reward
    agent.remember(observation, action, reward, observation_, done)
    #print(done)
    if not load_checkpoint:
      agent.learn()
    observation = observation_
  score_history.append(score)
  avg_score = np.mean(score_history[-100:])

  if avg_score > best_score:
    best_score = avg_score
    if not load_checkpoint:
      agent.save_models()

  print('episode ', i, 'score %.1f' % score,'avg score %1f' % avg_score )



....saving models ....
episode  0 score -1227.8 avg score -1227.835587
....saving models ....
episode  1 score -1630.1 avg score -1428.944027
....saving models ....
episode  2 score -1485.5 avg score -1447.789912
....saving models ....
episode  3 score -1416.7 avg score -1440.027429
....saving models ....
episode  4 score -1763.5 avg score -1504.719224
....saving models ....
episode  5 score -1734.0 avg score -1542.933120
....saving models ....
episode  6 score -1324.7 avg score -1511.763192
....saving models ....
episode  7 score -1273.8 avg score -1482.023759
....saving models ....
episode  8 score -754.8 avg score -1401.218750
....saving models ....
episode  9 score -1378.8 avg score -1398.979486
....saving models ....
episode  10 score -1025.8 avg score -1365.053707
....saving models ....
episode  11 score -1069.5 avg score -1340.422469
....saving models ....
episode  12 score -798.8 avg score -1298.755985
....saving models ....
episode  13 score -1008.4 avg score -1278.017116
....